In [0]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 17.0MB/s 
     |████████████████████████████████| 890kB 14.5MB/s 
     |████████████████████████████████| 3.8MB 25.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a4b88a02e707f0499948c3e9a750e77aadc3c2729a33aa6488e36f6c8c726e22
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import torch
import numpy as np

In [0]:
# from transformers import BertForQuestionAnswering, BertTokenizer, XLNetTokenizer, XLNetForQuestionAnswering, XLMTokenizer, XLMForQuestionAnswering
from transformers import BertForQuestionAnswering, BertTokenizer, DistilBertForQuestionAnswering, DistilBertTokenizer

# albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
# albert_model = AlbertForQuestionAnswering.from_pretrained('albert-base-v2')
# print("Albert loaded...")

# bert_large_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-mini-finetuned-squadv2')
# bert_large_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-mini-finetuned-squadv2')
# print("Bert Large loaded...")

# bert_base_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
# bert_base_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
# print("Bert Base loaded...")

# xlnet_tokenizer =  XLNetTokenizer.from_pretrained('xlnet-base-cased')
# xlnet_model = XLNetForQuestionAnswering.from_pretrained('xlnet-base-cased')

# xlm_tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
# xlm_model = XLMForQuestionAnswering.from_pretrained('xlm-mlm-en-2048')

bert_mini_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-mini-finetuned-squadv2')
bert_mini_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-mini-finetuned-squadv2')
print("Bert Mini loaded...")

bert_small_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
bert_small_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
print("Bert Small loaded...")

bert_med_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
bert_med_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
print("Bert Medium loaded...")

# distil_bert_model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')
# distil_bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
# print("DistilBert loaded...")


Bert Mini loaded...



Bert Small loaded...



Bert Medium loaded...


In [0]:
def ensemble_confidence_score(start_scores, end_scores):
  prob, answer_start, answer_end, model = 0, 0, 0, 0
  for i in range(len(start_scores)):
    start = np.argmax(start_scores[i])
    end = np.argmax(end_scores[i])
    if (start_scores[i][start]*end_scores[i][end]) > prob:
      prob = start_scores[i][start]*end_scores[i][end]
      model = i 
      answer_start = start 
      answer_end = end 

  return model, answer_start, answer_end

def ensemble_confidence_prob(start_prob, end_prob):
  prob, answer_start, answer_end, model = 0.0, 0, 0, 0
  for i in range(len(start_prob)):
    print("%f %f " %(max(start_prob[i]), max(end_prob[i])))
    for j in range(len(start_prob[i])):
      temp = start_prob[i][j] * end_prob[i]
      if max(temp) > prob:
        prob = max(temp)
        model = i
        answer_start = j
        answer_end = torch.argmax(torch.Tensor(temp))
    # print("%d %d %d %f" %(model, answer_start, answer_end, prob))

  return model, answer_start, answer_end
      

  # ans = torch.mm(torch.transpose(torch.Tensor(start_prob), 0, 1), torch.Tensor(end_prob))
  # print(ans)
  # return model, answer_start, answer_end

def ensemble_weighted_average(ranks, start_prob, end_prob):
  # use F1 ranks
  prob, answer_start, answer_end, model = 0.0, 0, 0, 0
  for i in range(len(start_prob)):
    for j in range(len(start_prob[i])):
      temp = ranks[i] * start_prob[i][j] * end_prob[i]
      if max(temp) > prob:
        prob = max(temp)
        model = i
        answer_start = j
        answer_end = torch.argmax(torch.Tensor(temp))
    # print("%d %d %d %f" %(model, answer_start, answer_end, prob))

  return model, answer_start, answer_end

In [0]:
def add_lists(tup):
  ans = tup[0]

  for i in range(1, len(tup)):
    np.add(ans, tup[i])

  return ans

def ensemble_weighted_average_new(ranks, start_prob, end_prob):
  prob, answer_start, answer_end, max_model, ans_val, max_start, max_end = [], 0, 0, 0, 0.0, 0, 0
  # number of models
  for i in range(len(start_prob)):
    # number of spans
    temp = []
    for j in range(len(start_prob[i])):
      t = ranks[i] * start_prob[i][j] * np.asarray(end_prob[i])
      temp.append(t)
      temp_val = max(t)
      if temp_val > ans_val:
        ans_val = temp_val 
        max_model = i
        max_start = j
        max_end = np.where(t == temp_val)
    
    prob.append(temp)

  # print(prob )
  answer_mat = np.asarray(prob[0])
  for i in range(1, len(prob)):
    answer_mat = np.add(answer_mat, np.asarray(prob[i]))

  # print(answer_mat)
  # print(answer_mat) 
  a = np.amax(answer_mat)
  answer_start = np.where(answer_mat == a)
  answer_end = int(answer_start[1][0])
  answer_start = int(answer_start[0][0])
    # print("%d %d %d %f" %(model, answer_start, answer_end, prob))

  return max_model, max_start, max_end, answer_start, answer_end

In [0]:
ranks = [float(0.5), float(0.5)]
start = [[1.0,2.0,3.0], [4.0,5.0,6.0]]
end = [[1.0,2.0,3.0], [4.0,5.0,6.0]]
max_model, max_start, max_end, answer_start, answer_end = ensemble_weighted_average_new(ranks, start, end)
print(max_model, max_start, max_end, answer_start, answer_end)

1 2 (array([2]),) 2 2


In [0]:
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

In [0]:
def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def compute_F1(a_gold_start, a_gold_end, a_pred_start, a_pred_end):
  print(a_gold_start, a_gold_end, a_pred_start, a_pred_end)
  num_same = min(a_gold_end, a_pred_end) - max(a_gold_start, a_pred_start) + 1
  gold_toks = list(range(a_gold_start, a_gold_end + 1))
  pred_toks = list(range(a_pred_start, a_pred_end + 1))
  if (a_gold_end-a_gold_start) == 0 or (a_pred_end-a_pred_start) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same <= 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [0]:
def guided_random_search(MAX_ITER, pred_start, pred_end, start_prob, end_prob):
  best_F1, best_weight = 0.0, []

  for i in range(MAX_ITER):
    w = np.rand(len(start_prob))
    model, answer_start, answer_end = ensemble_weighted_average(w, start_prob, end_prob)

    # calculate F1
    F1 = compute_F1(pred_start, pred_end, answer_start, answer_end)

    if F1 > best_F1:
      best_F1 = F1
      best_weight = w 

  return w 

def ensemble_guided_weighted_average(pred_start, pred_end, start_prob, end_prob):
  MAX_ITER = 10
  best_weight = guided_random_search(MAX_ITER, pred_start, pred_end, start_prob, end_prob)

  # for test
  return ensemble_weighted_average(best_weight, start_prob, end_prob)

In [0]:
question = "Who was Jim Henson?"
answer_text = "Jim Henson was a nice puppet" 

# ======== Tokenize ========
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids_x = bert_large_tokenizer.encode(question, answer_text)
input_ids_y = bert_base_tokenizer.encode(question, answer_text)

tokens_x = bert_large_tokenizer.convert_ids_to_tokens(input_ids_x)
tokens_y = bert_base_tokenizer.convert_ids_to_tokens(input_ids_y)

print('The input has a total of {:} tokens.'.format(len(input_ids_x)))
print('The input has a total of {:} tokens.'.format(len(input_ids_y)))

# ======== Set Segment IDs ========
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index_x = input_ids_x.index(bert_large_tokenizer.sep_token_id)
sep_index_y = input_ids_y.index(bert_base_tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a_x = sep_index_x + 1
num_seg_a_y = sep_index_y + 1

# The remainder are segment B.
num_seg_b_x = len(input_ids_x) - num_seg_a_x
num_seg_b_y = len(input_ids_y) - num_seg_a_y

# Construct the list of 0s and 1s.
segment_ids_x = [0]*num_seg_a_x + [1]*num_seg_b_x
segment_ids_y = [0]*num_seg_a_y + [1]*num_seg_b_y

# There should be a segment_id for every input token.
assert len(segment_ids_x) == len(input_ids_x)
assert len(segment_ids_y) == len(input_ids_y)

# ======== Evaluate ========
# Run our example question through the model.
start_scores_x, end_scores_x = bert_large_model(torch.tensor([input_ids_x]), # The tokens representing our input text.
                                token_type_ids=torch.tensor([segment_ids_x])) # The segment IDs to differentiate question from answer_text

                                # Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores_x)
answer_end = torch.argmax(end_scores_x)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens_x[answer_start:answer_end+1])

print('Answer_Bert_Large: "' + answer + '"')


start_scores_y, end_scores_y = bert_base_model(torch.tensor([input_ids_y]), 
                                token_type_ids=torch.tensor([segment_ids_y]))

# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores_y)
answer_end = torch.argmax(end_scores_y)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens_y[answer_start:answer_end+1])

print('Answer_Bert_Base: "' + answer + '"')

# print(start_scores_x, end_scores_x)
# print(start_scores_y, end_scores_y)

start = []
start.append(torch.nn.functional.softmax(start_scores_x, dim=1).detach().numpy()[0])
start.append(torch.nn.functional.softmax(start_scores_y, dim=1).detach().numpy()[0])
# start.append(start_scores_x.detach().numpy()[0])
# start.append(start_scores_y.detach().numpy()[0])

end = []
end.append(torch.nn.functional.softmax(end_scores_x, dim=1).detach().numpy()[0])
end.append(torch.nn.functional.softmax(end_scores_y, dim=1).detach().numpy()[0])
# end.append(end_scores_x.detach().numpy()[0])
# end.append(end_scores_y.detach().numpy()[0])

# print(start)

model, start_ind, end_ind = ensemble_confidence_prob(start, end)
# model, start_ind, end_ind = ensemble_confidence_score(start, end)


print("ANSWER: %d %d %d" %(model, start_ind, end_ind))
answer = ' '.join(tokens_x[start_ind:end_ind+1])
print(answer)

The input has a total of 14 tokens.
The input has a total of 14 tokens.
Answer_Bert_Large: "nice"
Answer_Bert_Base: "a nice puppet"
0.706727 0.605791 
0 11 11 0.428129
0.573222 0.664154 
0 11 11 0.428129
ANSWER: 0 11 11
nice


In [0]:
def answer_question(question, text, alpha):
  """
  Answering wrapper, change which function you use and which models you use it with
  """
  return ensemble_berts(bert_mini_model, bert_small_model, bert_med_model, bert_mini_tokenizer, bert_small_tokenizer, bert_med_tokenizer,
                        question, text, alpha)

In [0]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = "We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement)."

print(wrapper.fill(bert_abstract))

We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from Transformers. Unlike recent language
representation models (Peters et al., 2018a; Radford et al., 2018), BERT is
designed to pretrain deep bidirectional representations from unlabeled text by
jointly conditioning on both left and right context in all layers. As a result,
the pre-trained BERT model can be finetuned with just one additional output
layer to create state-of-the-art models for a wide range of tasks, such as
question answering and language inference, without substantial taskspecific
architecture modifications. BERT is conceptually simple and empirically
powerful. It obtains new state-of-the-art results on eleven natural language
processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute
improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1
question answering Test F1 to 93.2 (1.5 point absolute improvement) 

In [0]:
question = "What does the 'B' in BERT stand for?"
alpha = [0.33, 0.33, 0.33]

print(answer_question(question, bert_abstract, alpha ))

(26, 33)


In [0]:
question = "What are some example applications of BERT?"
alpha = [0.33, 0.33, 0.33]

print(answer_question(question, bert_abstract, alpha))

(129, 133)


In [0]:
def ensemble_berts(model_x, model_y, model_z, tokenizer_x, tokenizer_y, tokenizer_z, question, answer_text, alpha=[]):
    """
    Combines outputs from two transformers through a mean to determine the answer output.
    alpha parameter favors model_x over model_y ie.:
    alpha = 0.8 will give preference to model_x by 80%.
    Should be generic enough to work with Huggingface QA models.
    """
    max_len = 300
    # ======== Automated solution ========
    # Get token segment id's with encode plus 
    ## removed arg: pad_to_max_length=True
    input_dict_x = tokenizer_x.encode_plus(question, answer_text, max_length=max_len, pad_to_max_length=True, return_tensors='pt')
    input_dict_y = tokenizer_y.encode_plus(question, answer_text, max_length=max_len, pad_to_max_length=True, return_tensors='pt')
    input_dict_z = tokenizer_z.encode_plus(question, answer_text, max_length=max_len, pad_to_max_length=True, return_tensors='pt')

    # ======== Evaluate ========
    # Run our example question through the model.
    try:
      if len(input_dict_x)>0:
        start_scores_x, end_scores_x = model_x(**input_dict_x)
      else:
        return 0,0
    except:
      return 0, 0

    try:
      if len(input_dict_y)>0:
        start_scores_y, end_scores_y = model_y(**input_dict_y)
      else:
        return 0,0
    except:
      return 0,0 

    try: 
      if len(input_dict_z)>0:
        start_scores_z, end_scores_z = model_z(**input_dict_z)
      else:
        return 0,0
    except:
      return 0, 0

    start = []
    start.append(torch.nn.functional.softmax(start_scores_x, dim=1).detach().numpy()[0])
    start.append(torch.nn.functional.softmax(start_scores_y, dim=1).detach().numpy()[0])
    start.append(torch.nn.functional.softmax(start_scores_z, dim=1).detach().numpy()[0])
    # start.append(start_scores_x.detach().numpy()[0])
    # start.append(start_scores_y.detach().numpy()[0])
    # start.append(start_scores_z.detach().numpy()[0])

    end = []
    end.append(torch.nn.functional.softmax(end_scores_x, dim=1).detach().numpy()[0])
    end.append(torch.nn.functional.softmax(end_scores_y, dim=1).detach().numpy()[0])
    end.append(torch.nn.functional.softmax(end_scores_z, dim=1).detach().numpy()[0])
    # end.append(end_scores_x.detach().numpy()[0])
    # end.append(end_scores_y.detach().numpy()[0])
    # end.append(end_scores_z.detach().numpy()[0])

    # model, start_ind, end_ind = ensemble_confidence_prob(start, end)
    # model, start_ind, end_ind = ensemble_confidence_score(start, end)


    # print("ANSWER: %d %d %d" %(model, start_ind, end_ind))

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    model = 2
    model, max_start, max_end, answer_start, answer_end = ensemble_weighted_average_new(alpha, start, end)
    # answer = ""
    # answer = tokenizer_x.decode(input_dict_z['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)

    # answer = ''
    # if model == 0:
    #   answer = tokenizer_x.decode(input_dict_x['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    # elif model == 1:
    #   answer = tokenizer_y.decode(input_dict_y['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    # else:
    #   answer = tokenizer_z.decode(input_dict_z['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    
    # if not answer:
    #   answer_start, answer_end = 0, 0
    #   answer = ""
    # print(answer_start, answer_end, answer)
    return answer_start, answer_end, model 

In [0]:
#COLAB DRIVE MOUNT
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from transformers.data.processors.squad import SquadV2Processor

processor = SquadV2Processor()
eval_examples = processor.get_dev_examples("/content/gdrive/My Drive/squad")

100%|██████████| 35/35 [00:04<00:00,  7.70it/s]


In [0]:
import json
import string
import re
import collections
from google.colab import output

def run_predictions(eval_set):
  indices_start, indices_end, indices_text = [], [], []
  
  for i, example in enumerate(eval_set):
    a, b, c = [], [], []
    for answer in example.answers:
      a.append(answer['answer_start'])
      b.append(answer['answer_start']+len(answer['text'])-1)
      c.append(answer['text'])
    indices_start.append(a)
    indices_end.append(b)
    indices_text.append(c)

    # print("Run preds ...")
  print("Run preds done")
  # print(indices_start)

  return indices_start, indices_end, indices_text

def main_func(eval_examples):
  indices_start, indices_end, indices_text = run_predictions(eval_examples)
  best = []

  for i in range(2):
    w = np.random.dirichlet(np.ones(3),size=1)[0]
    w.sort()
    print(w)
    answer_start, answer_end, answer_text = [], [], []
    for j, example in enumerate(eval_examples):
      c = ""
      a, b , model = ensemble_berts(bert_mini_model, bert_small_model, bert_med_model, bert_mini_tokenizer, bert_small_tokenizer, bert_med_tokenizer,
                          example.question_text, example.context_text, w )
      
      if model == 0:
        input_dict_x = bert_mini_tokenizer.encode_plus(example.question_text, example.context_text, max_length=300, pad_to_max_length=True, return_tensors='pt')
        if bert_mini_tokenizer.decode(input_dict_x['input_ids'][0, a:b+1], skip_special_tokens=True):
          c = bert_mini_tokenizer.decode(input_dict_x['input_ids'][0, a:b+1], skip_special_tokens=True)
      elif model == 1:
        input_dict_x = bert_small_tokenizer.encode_plus(example.question_text, example.context_text, max_length=300, pad_to_max_length=True, return_tensors='pt')
        if bert_small_tokenizer.decode(input_dict_x['input_ids'][0, a:b+1], skip_special_tokens=True):
          c = bert_small_tokenizer.decode(input_dict_x['input_ids'][0, a:b+1], skip_special_tokens=True)
      else:
        input_dict_x = bert_med_tokenizer.encode_plus(example.question_text, example.context_text, max_length=300, pad_to_max_length=True, return_tensors='pt')
        if bert_med_tokenizer.decode(input_dict_x['input_ids'][0, a:b+1], skip_special_tokens=True):
          c = bert_med_tokenizer.decode(input_dict_x['input_ids'][0, a:b+1], skip_special_tokens=True)

      # print(j, a, b)
      if j%100 == 0:
        print(j)

      answer_start.append(a)
      answer_end.append(b)
      # print(c)
      answer_text.append(c)

    # print(answer_text)
    print("Done.... %d" %i)
    temp, best_F1 = 0.0, 0.0
    for k in range(len(answer_text)):
      t = 0.0
      for p in range(len(indices_text[k])):
        # t = max(t, compute_f1(indices_start[k][p], indices_end[k][p], answer_start[k], answer_end[k]))
        t = max(t, compute_f1(indices_text[k][p], answer_text[k]))
      temp += t

    print("temp %f" %temp)
    temp = float(temp / float(len(answer_text)))
    print("temp %f" %temp)
    if temp > best_F1:
      best_F1 = temp
      best = w

  print(best)   


In [0]:
print(len(eval_examples))

11873


In [0]:
main_func(eval_examples)

Run preds done
[0.02968807 0.22327263 0.74703931]
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700


In [0]:
import re
from dateutil.parser import parse
import stanfordnlp


def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

# load_data
stanfordnlp.download('en')
nlp = stanfordnlp.Pipeline()

answer_types = {}
for i in answers:
  if bool(re.search(r'-?\d+', inputString)):
    if is_date(inputString, fuzzy=True):
      answer_types['date'].append(i)
    else:
      answer_types['other_num'].append(i)
  else:
    doc = nlp(inputString)
    for sent in doc.sentences:
      for word in sent.word:
        if word.upos == "PROPN":
          ## do NER
        else:
          



SyntaxError: ignored